# Long Term Potentials

**Tensorflow**

We're gathering quite a lot of data about how people bid, the end values of lots given the qualification price, average bid rate, optimum lot length and various other metrics, purely by virtue of retaining all the events in the database.

I think there's an opportunity here to give advice to the Hosts about how they should structure their auctions given their requirements. Exactly how we go about that is a question of its own, but given some specific questions ("what lot length results in the most bids without long waits at the beginning or end, and without cutting off a good competition?" would be a naive example) we could train a TensorFlow neural net on the data we already have and see if it produces good predictions which we could then feed back to the Hosts and help them design their lots better.

**Natural Language Processing**

It would seem reasonable that, given we already provide document uploading capabilities, we could wire the document retrieval to the Google Natural Language API. That would allow all participants and hosts to submit and retrieve documents in their own language, without worrying about to whom they will be sent or in what language they were originally. For companies like Aggreko, who could have someone speaking one language submitting jobs to be bid for in countries where the official language is something different, this would be a significant reduction in friction. Currently the most 'production ready' languages are English and Spanish, which would be somewhat useful in Europe and moreso in the Americas.

In this vein we should encourage the people who spend most time on support (Lewis, Pete, Craig) to use the web interface for support, as Zoho provide a realtime translation service. It looks good to look multinational and talk to people in their own languages.

**Amazon Scaleability**

Amazon provide both database instances (RDS) and compute (EC2), plus storage (S3). Conveniently packaged as Elastic Beanstalk this provides us somewhere to put the app which will autoscale (to a point). As a place to move away from Liberata this offers significant potential.

**Simian Army**

If we're bringing on bigger clients, and we need to provide things like APIs and service assurances, we will need to (at minimum) mirror the application. No need to shard the database unless we get significantly more business, but the app itself ought to have multiple instances and load balancing. Doing this with Kubernetes (or Amazon's equivalent containerised EC2 service) makes the most sense (see Amazon Scaleability), and once we get a good load balancer in front of it should make the app much more resilient.

As part of this, it would be a good idea to start introducing Simian Army. Janitor Monkey in particular would be helpful; Conformity Monkey is essentially obviated by the use of containerised environments and we don't have the staff to cope with Chaos Monkey yet (although that would be a good plan for the longer term).

**Rspec Deprecations**

Long term because this is going to be a big job. We need to sort out the deprecation issues, whether that's changing some syntax or upgrading some gems. They're slowing us down, not to mention driving the devs insane because running tests generates a sea of rubbish. They also may partly be behind why running Rspec in parallel fails. 

**Ruby Deprecations**

Disclosed by Rspec, Test::Unit and various other tools. Same issue.

**Gem Deprecations**

These will be harder to find, and some may only appear once we upgrade our Rails version and discover old gems don't work. Comatose in particular is definitely end-of-life software, but there may well be others.

**Rubocopping**

The automated stuff we can do immediately, that's not a problem, although some of the automatic cops aren't very bright and break things. Once that's done, though, the rest need manually changing. This is especially important if we want to upgrade the Rails version, since that's going to break many, many things further down the line and it's hard to tell what the real problem is if some of it is syntax errors. See also deprecations.

**Test Conversion**

A number of the tests for the app are still in Test::Unit rather than Rspec, and those that are in Rspec are purely for the Ruby. Converting all the Ruby tests to Rspec, and then creating a suite of tests for our JS with Jasmine (which, according to GitHub, is actually larger than the Ruby code!) which Rspec runs when commanded, should give us much better clarity on exactly what is working and what is not. Might even uncover some bugs. 

We also have some issues (many issues!) with running tests late at night and early in the morning. Optimally these would be fixed; it appears to be an issue with Timecop. Rspec also generates a large number of failures when running in parallel; even with the necessary multiple test databases and environment variables. This means we have inter-test dependencies, which need to be addressed for true unit testing to work. We should be running all tests in parallel and in randomised order.

**Moving PM Tools**

Moving from Zoho bugtracker to GitHub issues and Kanban boards

**Jenkins Test Coverage Task**

Fix the metric_fu task on Jenkins so that we have a good grasp on our code coverage on an ongoing basis.
Have Jenkins POST the result to (somewhere?) for storage and review.

**Performance Metrics**

Automated testing of pageload times for the site.

**TODO Cleanup of codebase**
Find all TODO, NOTE etc comments (including those which should be but aren't!), check if they have been assigned issues on the issue tracker, date and attribute them if possible.

**Documentation**

Connections between .erb templates, .js scripts and .rb backend should be better (or at all!) documented. Current situation requires searching the entire .js corpus for the id or class attribute of the element that one is looking at, evaluating which of the results looks likely, and then changing small pieces to see what happens.

**Tooling**

Foreman? Webpack?

**Address Issue of Documentation Review Periods**

Documentation should be evaluated and updated at the beginning of each quarter.

Dependencies (such as gems, especially those with native dependencies such as Nokogiri) should be evaluated and updated (if necessary and possible given compatibility constraints) at the beginning of each half.

Core modules (Rails, CMS) should be evaluated and updated at each minor (x.**X**.x.x) and major (**X**.x.x.x) release, to prevent multi-step upgrades from being necessary in future.

**Ideas**

 - In-memory db; more aggressive caching
  - Redis for commonly-accessed info?
  

 - Concurrent access
 - Duplicate/mirror production DB on separate server for testing purposes; at most five minutes old.
  - **One way** transactions, obviously.
 
 - Duplicate open issues on Zoho to the GitHub issues board; easier to keep track of everything in one place.
  - Also allows planning via GitHub Kanban board, which can be linked directly to issues, commits, releases and milestones.
 
 - User licence data in database
  - Ending dates
  - Invoiced?
  - Paid?
 
 - Single Sign On (Using OmniAuth - Google probably best provider)
  - Definitely for SQL Panel
  - Potentially for main app?
  
 - API for data retrieval
  - Maybe not part of the Rails app; it's already huge. 
  - Small Go service that responds to HTTP request? 
   - Advantage of not needing views
   - Lower overheads
   - Read-only access ensured by separate DB user (maybe use the same as the SQL Panel)
  - Alternative: add JSON return option to the SQL Panel, treat it as the overall data retrieval service

 - API for auction creation
  - MAYBE. Requires write access which is dangerous to have on an API.
 
 - Contract storage integration

 - Wheneverize
 
 - Tests depend on each other?!
 - Negative time returned on some TestUnit tests
 - phs_report.rb needs changing (including name)
 - Might be the cause of the test failures at midnight
 
 - Moving away from Liberata (target 2016-12)
  - IL3 (minimum) or CESG security level accreditation ideal
  - G-Cloud accreditation required
 
 - Move registration email signup to Mailchimp from Zoho
 - Why do we use Blogger for the blog? Could we swap it out for Ghost without breaking anything?
 
 - Script to accept Positionly data as CSV, remove lines with >100 cells in (no mentions), aggregate and graph results over time.
 
 - Add Blazer to MD to allow everyone to run SQL reports against the live database
  - Use reports from Lewis to start off with.
  - Save SQL queries run by me and Nic in order to provide prebuilt ones for everyone else.
  - Impossible as part of monolith due to Rails version; microservice in progress
   - Use this as a starting point for duplicating app entirely, call it the R5 version...?
  - Devise needed for security; again, useful for the new version of the app.
   - Currently in place, but need to copy users data into the migration from MD before migrating. 
   - Create entirely new database. Let's begin again; we have proved we can use Blazer to pull data.
   - Very slow replication of old app. Rubocop every file when added, fix everything possible.
  
 - Blog post on digital nomadism
 
 - Post-R5 move, change server to Puma and web server to Caddy (speed, ease of use)
 
 - Document dependencies and their purposes, especially where we are using forks or direct GitHub links.

 - Add numeric output for automated `rake test`, `rubocop`, `rspec` and `reek` testing to Jenkins; try to avoid having tens of thousands of warnings by having a human look at those. Automated filters can only take us so far.

 - Investigate all deprecation warnings produced by the various runners. Those which can be fixed before moving up a Rails or Ruby version, fix; that will make the upgrade process considerably easier.